# Import Modules

In [1]:
import os
print(os.getcwd())
import sys


# #########################################################
from methods import (
    get_df_dft,
    get_df_jobs,
    get_df_jobs_data,
    get_df_slab,

    convert_str_facet_to_list,
    )

from local_methods import create_slab_from_bulk_tmp, create_final_slab_master_tmp

/mnt/f/Dropbox/01_norskov/00_git_repos/PROJ_IrOx_OER/workflow/creating_slabs/creating_symm_slabs


# Read Data

In [2]:
df_dft = get_df_dft()

df_jobs = get_df_jobs()

df_jobs_data = get_df_jobs_data()

df_slab = get_df_slab()

In [3]:
# Only taking systems which are being calculated (or finished) via DFT
# df_slab_i = df_slab.loc[df_jobs.slab_id.unique().tolist()]

slabs_dft_finished = df_jobs_data[df_jobs_data.completed == True].slab_id.tolist()
df_slab_i = df_slab.loc[slabs_dft_finished]

In [4]:
df_slab_i = df_slab_i[df_slab_i.bulk_id == "cq7smr6lvj"]

In [5]:
df_slab_i

,slab_id,bulk_id,facet,slab_final,num_atoms,loop_time,iter_time_i
slab_id,,,,,,,
sekivema_89,sekivema_89,cq7smr6lvj,100,"(Atom('O', [3.033379352999999, 3.5966878298999...",54,0.016979,56.744781


In [6]:
for slab_id, row_i in df_slab_i[["bulk_id", "facet", ]].iterrows():
    tmp = 42


    # #####################################################
    bulk_id = row_i.bulk_id
    facet = row_i.facet
    # #####################################################

    # #####################################################
    row_dft_i = df_dft.loc[bulk_id]
    # #####################################################
    bulk_atoms = row_dft_i.atoms
    # #####################################################

    facet_list = convert_str_facet_to_list(facet)

    slab_0 = create_slab_from_bulk_tmp(
        atoms=bulk_atoms, facet=facet_list)

    # slab_1 = create_final_slab_master(atoms=slab_0)

    # slab_2 = constrain_slab(atoms=slab_1)
    # slab_final = slab_2

    # df_coord_slab_final = get_structure_coord_df(slab_final)

    file_name_i = "temp_" + bulk_id + "_" + slab_id + ".cif"
    file_path_i = os.path.join(
        "out_data",
        file_name_i)
    # slab_0.write("out_data/temp.cif")
    slab_0.write(file_path_i)

In [7]:
# slab_1 = create_final_slab_master_tmp(atoms=slab_0)

In [ ]:
# remove_all_atoms_above_cutoff_symm

atoms = slab_0

# Developing new symmetric surface cutter

In [8]:
#| - Import Modules
import os
import sys

import copy
from pathlib import Path

import json

import numpy as np
import pandas as pd

# #############################################################################
from ase import io

from pymatgen.analysis.chemenv.coordination_environments.coordination_geometry_finder import LocalGeometryFinder
from pymatgen.analysis.chemenv.coordination_environments.chemenv_strategies import (
    SimplestChemenvStrategy,
    MultiWeightsChemenvStrategy)
from pymatgen.analysis.chemenv.coordination_environments.structure_environments import LightStructureEnvironments
from pymatgen.io.ase import AseAtomsAdaptor

from catkit.gen.surface import SlabGenerator

# #############################################################################
from methods import (
    get_df_dft, symmetrize_atoms,
    get_structure_coord_df, remove_atoms)
from proj_data import metal_atom_symbol
#__|

from local_methods import (
    remove_all_atoms_above_cutoff,
    calc_surface_area,
    remove_highest_metal_atoms,
    get_slab_thickness,
    create_slab_from_bulk_tmp,
    remove_noncoord_oxygens,
    remove_nonsaturated_surface_metal_atoms,
    )

In [9]:
# def create_final_slab_master_tmp(
atoms=slab_0
#     ):
#     """Master method to create final IrOx slab.
#     """

In [10]:
# def create_final_slab_master_tmp(
#     atoms=None,
#     ):
#     """Master method to create final IrOx slab.
#     """
#| - create_final_slab_master
TEMP = True
slab = atoms

###########################################################
slab_thickness_i = get_slab_thickness(atoms=slab)
# print("slab_thickness_i:", slab_thickness_i)


# slab = remove_all_atoms_above_cutoff(atoms=slab, cutoff_thickness=17)
slab = remove_all_atoms_above_cutoff_symm(atoms=slab, cutoff_thickness=17)
if TEMP:
    slab.write("out_data/temp_out/slab_1_0.cif")

slab_thickness_i = get_slab_thickness(atoms=slab)
# print("slab_thickness_i:", slab_thickness_i)


###########################################################
slab = remove_nonsaturated_surface_metal_atoms(
    atoms=slab,
    dz=4)
if TEMP:
    slab.write("out_data/temp_out/slab_1_1.cif")

# slab.write("out_data/temp_2.cif")
slab_thickness_i = get_slab_thickness(atoms=slab)
# print("slab_thickness_i:", slab_thickness_i)

slab = remove_noncoord_oxygens(atoms=slab)
if TEMP:
    slab.write("out_data/temp_out/slab_1_2.cif")

slab_thickness_i = get_slab_thickness(atoms=slab)
# print("slab_thickness_i:", slab_thickness_i)



###########################################################
i_cnt = 2
while slab_thickness_i > 15:
    i_cnt += 1
    # print(i_cnt)

    # #####################################################
    # Figuring out how many surface atoms to remove at one time
    # Taken from R-IrO2 (100), which has 8 surface Ir atoms and a surface area of 58 A^2
    surf_area_per_surface_metal = 58 / 8
    surface_area_i = calc_surface_area(atoms=slab)
    ideal_num_surface_atoms = surface_area_i / surf_area_per_surface_metal
    num_atoms_to_remove = ideal_num_surface_atoms / 3
    num_atoms_to_remove = int(np.round(num_atoms_to_remove))
    # #####################################################

    slab_new_0 = remove_highest_metal_atoms(
        atoms=slab,
        num_atoms_to_remove=num_atoms_to_remove,
        metal_atomic_number=77)
    if TEMP:
        slab_new_0.write("out_data/temp_out/slab_1_" + str(i_cnt) + "_0" + ".cif")

    slab_new_1 = remove_nonsaturated_surface_metal_atoms(
        atoms=slab_new_0,
        dz=4)
    if TEMP:
        slab_new_1.write("out_data/temp_out/slab_1_" + str(i_cnt) + "_1" + ".cif")

    slab_new_2 = remove_noncoord_oxygens(atoms=slab_new_1)
    if TEMP:
        slab_new_2.write("out_data/temp_out/slab_1_" + str(i_cnt) + "_2" + ".cif")


    slab_thickness_i = get_slab_thickness(atoms=slab_new_2)
    # print("slab_thickness_i:", slab_thickness_i)

    if TEMP:
        slab_new_2.write("out_data/temp_out/slab_1_" + str(i_cnt) + ".cif")

    slab = slab_new_2

# return(slab)
#__|

In [11]:
slab_1 = slab

In [12]:
slab_1.write("out_data/temp_1.cif")

In [13]:
# from catkit.gen import symmetry

# # symmetry.g